In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType, IntegerType, ShortType
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col, regexp_replace, sha2, when, lit, uuid

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "clientes"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_cliente"
TABELA_INVALIDOS_DESTINO = "tb_cliente_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

timestamp_atual = F.current_timestamp()


In [0]:
cliente_bronze_df = spark.read.table("v_credit.bronze.clientes")

EMAIL_REGEX = r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$"

cliente_limpos_df = (
    cliente_bronze_df
    .select(
        F.uuid().alias("cd_cliente").cast(StringType()), 

        F.col("nome").alias("nm_cliente").cast(StringType()),
        F.col("regiao").alias("nm_regiao").cast(StringType()),
        F.col("idade").alias("nu_idade").cast(ShortType()),
        F.col("ingestion_timestamp").alias("dt_ingestion").cast(TimestampType()),
        F.col("origem").alias("dc_origem").cast(StringType()),

        regexp_replace(F.col("email"), r'\+[A-Za-z0-9._%+-]+@', "@").alias("ds_email"),
        
        sha2(F.col("id_cliente").cast(StringType()), 256).alias("nu_cpf"),

        when(F.col("email").rlike(EMAIL_REGEX), lit(True)).otherwise(lit(False)).alias("flag_email_valid"),
        when(F.col("nu_idade") >= 18, lit(True)).otherwise(lit(False)).alias("flag_maior_de_idade")
    )
)

In [0]:
cliente_validacao = (
    cliente_limpos_df
    .withColumn(
        "flag_qualidade",
        when(
            (col("flag_email_valid") == True) & (col("flag_maior_de_idade") == True),
            lit("OK")
        ).otherwise(lit("ERRO"))
    )
)

cliente_validado = cliente_validacao.filter(col("flag_qualidade") == "OK")
cliente_invalido = cliente_validacao.filter(col("flag_qualidade") == "ERRO")


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, full_table_name_destination)

delta_table.alias("target").merge(
    cliente_validado.alias("source"),
    "target.nu_cpf = source.nu_cpf"
).whenMatchedUpdate(set={
    "nm_regiao": "source.nm_regiao",
    "nu_idade": "source.nu_idade",
    "dt_ingestion": "source.dt_ingestion",
    "dc_origem": "source.dc_origem"
}).whenNotMatchedInsertAll().execute()

print(f"✅ MERGE INTO executed successfully on table {full_table_name_destination}!")

In [0]:
cliente_invalido.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}")

In [0]:
tb_cliente_df = spark.read.table("v_credit.silver.tb_cliente")

duplicados_df = (
    tb_cliente_df
    .groupBy("nu_cpf", "cd_cliente")
    .count()
    .filter(col("count") > 1)
)

display(duplicados_df)